# Домашнее задание: декораторы

## Импорт библиотек, установка констант

In [3]:
import requests
import time
import re

from functools import wraps
from random import randint

In [2]:
BOOK_PATH = 'https://www.gutenberg.org/files/2638/2638-0.txt'

## Задание 1

In [129]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло выполнение декорируемой функции
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        res = func(*args, **kwargs)
        end = time.perf_counter()
        print(f'Время выполнения функции {func.__name__}: {end - start}')
        return res

    return wrapper

## Задание 2

In [130]:
def logging(func):
    """
    Декоратор, который выводит параметры с которыми была вызвана функция
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        print(f'Функция вызвана с параметрами: \n{args}, {kwargs}')
        return res

    return wrapper

## Задание 3

In [131]:
def counter(func):
    """
    Декоратор, считающий и выводящий количество вызовов декорируемой функции
    """
    c = 0

    @wraps(func)
    def wrapper(*args, **kwargs):
        nonlocal c
        res = func(*args, **kwargs)
        c += 1
        print(f'Функция была вызвана: {c} раз')
        return res
    
    return wrapper

## Задание 4

In [132]:
def memo(func):
    """
  Декоратор, запоминающий результаты исполнения функции func, чьи аргументы args должны быть хешируемыми
  """
    cache = {}

    @wraps(func)
    def fmemo(*args):
        for arg in args:
            if arg not in cache:
                cache[arg] = func(arg)
            return cache[arg]

    #fmemo.cache = cache
    return fmemo

## Тестирование

In [133]:
@counter
@logging
@benchmark
def word_count(word, url=BOOK_PATH):
    """
    Функция для посчета указанного слова на html-странице
    """

    # отправляем запрос в библиотеку Gutenberg и забираем текст
    raw = requests.get(url).text

    # заменяем в тексте все небуквенные символы на пробелы
    processed_book = re.sub('[\W]+' , ' ', raw).lower()

    # считаем
    cnt = len(re.findall(word.lower(), processed_book))

    return f"Cлово {word} встречается {cnt} раз"

print(word_count('whole'))

Время выполнения функции word_count: 2.0264716999990924
Функция вызвана с параметрами: 
('whole',), {}
Функция была вызвана: 1 раз
Cлово whole встречается 176 раз


In [134]:
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [135]:
%%timeit -n 10
fib(30)

138 ms ± 2.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [136]:
@memo
def fib(n):
    if n < 2:
        return n
    return fib(n-2) + fib(n-1)

In [137]:
%%timeit -n 10
fib(30)

The slowest run took 11.48 times longer than the fastest. This could mean that an intermediate result is being cached.
733 ns ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
